In [16]:
from typing import TypeVar

import sqlalchemy as sa
from sqlalchemy import NullPool

import pandas as pd

In [17]:
SelfClickHouseConnection = TypeVar("SelfClickHouseConnection", bound="ClickHouseConnection")

class ClickHouseConnection:
    """Класс контекстного менеджера для работы с ClickHouse"""
    
    def __init__(self, host: str, username: str, password: str, port: str = "9000"):
        """
        Параметры:
            host: хост ClickHouse
            username: имя пользователя ClickHouse
            password: пароль пользователя ClickHouse
            port: порт ClickHouse
        """
        
        self.session = None
        self.__host = host
        self.__port = port
        self.__username = username
        self.__password = password
        
    def __get_url(self) -> str:
        """Метод формирует connection string для подключения к ClickHouse"""
        
        url = "clickhouse+native://{username}:{password}@{host}:{port}"
        url = url.format(
            host = self.__host,
            port = self.__port,
            username = self.__username,
            password = self.__password
        )
        return url
        
    def __enter__(self: SelfClickHouseConnection) -> SelfClickHouseConnection:
        engine = sa.create_engine(self.__get_url(), poolclass=NullPool)
        self.session = engine.connect()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.session.close()
        
    def read_sql(self, query: str) -> pd.DataFrame:
        """
        Запрос к БД через фреймворк pandas
        
        Параметры:
            query: SQL в формате строки
            
        Возвращает:
            датафрейм с результатами запроса
        """
        
        with self as connector:
            return pd.read_sql(query, con=connector.session)

In [18]:
host = "10.120.1.11"
username = "amarbuliev"
password = "KuUNgVlPQiSN6dRqk"

connection_manager = ClickHouseConnection(host=host, username=username, password=password)

In [26]:
df = connection_manager.read_sql("SELECT * FROM system.tables")

In [20]:
pd.set_option('display.max_rows', None)

In [21]:
df

,database,name,uuid,engine,is_temporary,data_paths,metadata_path,metadata_modification_time,dependencies_database,dependencies_table,...,active_parts,total_marks,lifetime_rows,lifetime_bytes,comment,has_own_data,loading_dependencies_database,loading_dependencies_table,loading_dependent_database,loading_dependent_table
0,INFORMATION_SCHEMA,COLUMNS,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
1,INFORMATION_SCHEMA,KEY_COLUMN_USAGE,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
2,INFORMATION_SCHEMA,REFERENTIAL_CONSTRAINTS,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
3,INFORMATION_SCHEMA,SCHEMATA,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
4,INFORMATION_SCHEMA,TABLES,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
5,INFORMATION_SCHEMA,VIEWS,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
6,INFORMATION_SCHEMA,columns,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
7,INFORMATION_SCHEMA,key_column_usage,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
8,INFORMATION_SCHEMA,referential_constraints,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
9,INFORMATION_SCHEMA,schemata,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]


In [28]:
connection_manager.read_sql("SELECT * FROM sandbox.ongoing_projects")

,id,status,statusDesc,nameRus,head,directionHead,type,typeDesc,typeId,statusId,...,payed,projectOfficeControl,studentsCount,searchString,clientLogo,detailed_team,projectIndustryLabel,leaders,initiatorId,thumbnail
0,7,control_passed,Рабочий,Разработка нового модуля для системы корпорати...,Сергеев Антон Валерьевич,Сергеев Антон Валерьевич,soft,Неизвестно,2,2,...,True,True,8,Сергеев Антон Валерьевич 19032 Разработка ново...,,"[{'fullName': 'Гришкина Валерия Игоревна', 'na...",ИКТ,"[{'id': 7, 'fio': 'Башун Владимир Владимирович...",NaN,https://cabinet.miem.hse.ru/project/thumbnail/7
1,11,control_passed,Рабочий,Smart медицина,Фимина Ксения Игоревна,Белов Александр Владимирович,soft,Неизвестно,2,2,...,True,True,5,Фимина Ксения Игоревна 19058 Smart медицина Па...,,"[{'fullName': 'Гончаров Никита Павлович', 'nam...",Здравоохранение,"[{'id': 67, 'fio': 'Фимина Ксения Игоревна', '...",67.0,nan
2,17,control_passed,Рабочий,Оснащение Медиацентра МИЭМ,Елисеенко Андрей Михайлович,Королев Денис Александрович,soft-hard,Неизвестно,3,2,...,True,True,4,Елисеенко Андрей Михайлович 19102 Оснащение Ме...,,"[{'fullName': 'Асташов Степан Дмитриевич', 'na...",ИКТ,"[{'id': 633, 'fio': 'Елисеенко Андрей Михайлов...",NaN,nan
3,20,control_passed,Рабочий,Система бронирования аудиторий,Рыбаков Петр Владимирович,Рыбаков Петр Владимирович,soft,Неизвестно,2,2,...,True,True,2,Рыбаков Петр Владимирович 19105 Система бронир...,,"[{'fullName': 'Мартинич Андрей Иванович', 'nam...",ИКТ,"[{'id': 55, 'fio': 'Рыбаков Петр Владимирович'...",NaN,nan
4,21,control_passed,Рабочий,Mekstack: приватное облако,Емельяненко Максим Владимирович,Рыбаков Петр Владимирович,soft,Неизвестно,2,2,...,True,True,15,Емельяненко Максим Владимирович 19106 Mekstack...,,"[{'fullName': 'Автайкин Артём Олегович', 'name...",ИКТ,"[{'id': 1734, 'fio': 'Емельяненко Максим Влади...",NaN,https://cabinet.miem.hse.ru/project/thumbnail/21
5,24,control_passed,Рабочий,Цифровое портфолио,Сластников Сергей Александрович,Сластников Сергей Александрович,soft,Неизвестно,2,2,...,True,True,1,Сластников Сергей Александрович 19109 Цифровое...,,"[{'fullName': 'Дырченкова Юлия Александровна',...",Неизвестно,"[{'id': 62, 'fio': 'Сластников Сергей Александ...",NaN,nan
6,27,control_passed,Рабочий,Система управления проектами для проектной мод...,Шин Владимир Андреевич,Башун Владимир Владимирович,soft,Неизвестно,2,2,...,True,True,3,Шин Владимир Андреевич 19112 Система управлени...,,"[{'fullName': 'Александров Илья Витальевич', '...",ИКТ,"[{'id': 4089, 'fio': 'Шин Владимир Андреевич',...",NaN,https://cabinet.miem.hse.ru/project/thumbnail/27
7,28,control_passed,Рабочий,Распознавание медиаконтента,Моисеев Михаил Васильевич,Королев Денис Александрович,soft,Неизвестно,2,2,...,True,True,0,Моисеев Михаил Васильевич 19113 Распознавание ...,,[],ИКТ,"[{'id': 1385, 'fio': 'Моисеев Михаил Васильеви...",NaN,nan
8,44,control_passed,Рабочий,Система автоматизированного учета и выдачи мат...,Американов Александр Александрович,Романов Александр Юрьевич,soft-hard,Неизвестно,3,2,...,True,True,5,Американов Александр Александрович 19230 Систе...,,"[{'fullName': 'Абрамов Илья Александрович', 'n...",ИКТ,"[{'id': 1052, 'fio': 'Американов Александр Але...",NaN,nan
9,147,control_passed,Рабочий,Теоретическое исследование магнитной структуры...,Пугач Наталия Григорьевна,,nir,Неизвестно,1,2,...,True,True,2,Пугач Наталия Григорьевна 19659 Теоретическое ...,,"[{'fullName': 'Ильин Евгений Владимирович', 'n...",Наука,"[{'id': 51, 'fio': 'Пугач Наталия Григорьевна'...",NaN,nan
